In [547]:
import pandas as pd
import numpy as np
import import_ipynb
import sys
import math 
import seaborn as sea
import datetime as dt
import Fuzzy_Prediction_Beta as fz
from datetime import timedelta
from operator import itemgetter
from functools import partial
url = "in.csv"

In [548]:
dataset = pd.read_csv(url, error_bad_lines=False)

In [549]:
selected = dataset[['cod_location','date_id','total_cases']]

In [550]:
selected = selected.groupby('cod_location').tail(5)

In [551]:
selected.head()
#Input na rede neural
selected.to_csv('input.csv')

In [552]:
def aver(input):
    medias = input.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(5).mean())
    return medias

In [553]:
def diff(input):
    difference = input.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0] - x.head(1).values[0])
    return difference

In [554]:
def first(input):
    first = selected.groupby('cod_location')['total_cases'].apply(lambda x: x.head(1).values[0])
    return first

In [555]:
def last(input):
    last = selected.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0])
    return last

In [556]:
def param(last,diffe):
    param = last + diffe
    return param

In [557]:
def fuzzifyErrorsPos(erro, media):
    return fz.trapezio(erro, [(media + ((media * 25) / 100)), (media + ((media * 60) / 100)), (media + ((media * 80) / 100)), (media + ((media * 80) / 100))])

In [558]:
def fuzzifyErrorPos(erro, media):
    return fz.triangulo(erro, [(media + ((media * 8) / 100)), (media + ((media * 15) / 100)), (media + ((media * 30) / 100))])

In [559]:
def fuzzifyErrorZ(erro, media):
    return fz.trapezio(erro, [(media - ((media * 10) / 100)), (media - ((media * 5) / 100)), (media + ((media * 5) / 100)), (media + ((media * 10) / 100))])

In [560]:
def fuzzifyErrorNeg(erro, media):
    return fz.triangulo(erro, [(media - ((media * 30) / 100)), (media - ((media * 15) / 100)), (media - ((media * 8) / 100))])

In [561]:
def fuzzifyErrorsNeg(erro, media):
    return fz.trapezio(erro, [0, 0, (media - ((media * 60) / 100)), (media - ((media * 40) / 100))])

In [562]:
def verifRegras(erro, media):
    regras = [[0] * 5 for i in range(5)]
    
    fuzzifyErrorsNega = fuzzifyErrorsNeg(erro, media)
    fuzzifyErrorNega = fuzzifyErrorNeg(erro, media)
    fuzzifyErrorZe = fuzzifyErrorZ(erro, media)
    fuzzifyErrorPosi = fuzzifyErrorPos(erro, media)
    fuzzifyErrorsPosi = fuzzifyErrorsPos(erro, media)
    
    regras[0] = fuzzifyErrorsNega
    regras[1] = fuzzifyErrorNega
    regras[2] = fuzzifyErrorZe    
    regras[3] = fuzzifyErrorPosi    
    regras[4] = fuzzifyErrorsPosi
    
    return regras

In [563]:
media = aver(selected)
diffe = diff(selected)
first = first(selected)
last = last(selected)
param = last + diffe
maximus = max(selected['total_cases'])

In [564]:
fuzzy_data = pd.DataFrame()

In [565]:
## Códigos: 0-Queda Brusca 1-Queda 2-Platô 3-Subida 4-Subida Brusca
for i in range(212):
    Regras = verifRegras(param[i], media[i])
    prev = fz.getOutput(Regras)
    per  = fz.getPercentage(Regras)
    #centroide = fz.getCentroid(prev,Regras,per,maximus)
    fuzzy_data = fuzzy_data.append({'cd_local':i,'id_caso_prev':prev,'percent':per},ignore_index=True)
    fuzzy_data['cd_local'] = fuzzy_data['cd_local'].astype(int)
    fuzzy_data['id_caso_prev'] = fuzzy_data['id_caso_prev'].astype(int)
    

Fuzzy_Prediction_Beta.ipynb:3: RuntimeWarning: divide by zero encountered in double_scalars
  {
Fuzzy_Prediction_Beta.ipynb:9: RuntimeWarning: invalid value encountered in double_scalars
  "import pandas as pd\n",
Fuzzy_Prediction_Beta.ipynb:11: RuntimeWarning: invalid value encountered in double_scalars
  "import sys\n",


In [566]:
fuzzy_data.to_csv('predict.csv')

In [567]:
fuzzy_data

,cd_local,id_caso_prev,percent
0,0,2,100.00
1,1,2,92.89
2,2,2,100.00
3,3,2,71.09
4,4,0,90.64
...,...,...,...
207,207,2,100.00
208,208,2,91.45
209,209,2,100.00
210,210,2,100.00
